In [1]:
import spacy
from spacy.tokenizer import Tokenizer
import nltk.data
import nltk
from nltk import word_tokenize
import re
from nltk import FreqDist
import pandas as pd
import re
import csv
import pickle
import itertools
import glob
#!pip install duplicates
#!pip install iteration-utilities
from iteration_utilities import duplicates
from iteration_utilities import unique_everseen

### Carregando dados de texto a serem rotulados

In [3]:
#Carregando os mesmos textos que foram utilizados para a rotulação manual

dados_De_Texto_Para_Rotular = []
with open("C:\\Users\prisc\\Downloads\\PROJETO_MESTRADO\\Atual_Codigo_Para_Rotulacao_SemiAutomatica\\Dados_Utilizados_Para_RotulacaoManual.txt", "r",encoding='utf-8') as fp:
    list_Lines = fp.readlines()
    for line in list_Lines:
        dados_De_Texto_Para_Rotular.append(line.replace('\n',''))

#for item in dados_De_Texto_Para_Rotular:
    #print(item)

### Carregando os dicionários que contém as entidades versus categorias

In [17]:
# 1 - Cria uma lista com os endereços dos arquivos no formato de dicionario. Dicionarios possuem que vários elementos...
# a chave é a entidade (texto) e o valor a categoria desse elemento
# 2 - Um laço for é utilizado para percorrer essa lista de endereços
# 3 - E a cada volta do laço um endereço é passado para o método with open() e, o arquivo binário do referido endereço é aberto
# 4 - E o conteúdo do arquivo (um diciário com vários elementos) é inserido na lista de dicionarios.
# Desta maneira teremos uma lista composta por vários dicionarios. E vários dicionários que possuem vários itens. E os itens...
# são as entidades versus categorias, que o algoritmos de rotulação semiautomatica precisa para rotular os textos.

files_Dicionario = glob.glob ('C:\\Users\\prisc\\Downloads\\PROJETO_MESTRADO\\Entidades_E_Categorias_Para_Usar_No_Algoritmo_De_RotulacaoSemiAutomatica\*.txt')

lista_Dicionarios = []

for index, file in enumerate(files_Dicionario):
    with open(file, "rb") as fp:
        lista_Dicionarios.append(pickle.load(fp))

print("Total de itens - Entidades x Categorias:", len(lista_Dicionarios[0]))
#lista_Dicionarios

Total de itens - Entidades x Categorias: 2906


In [8]:
# 5 - Cria um único dicionario com todos os dicionarios que estam em diferentes posições da lista de dicionarios.

dicionario_EntidadeCategoria = {}

for item in lista_Dicionarios :
    dicionario_EntidadeCategoria.update(item)
    #print(dicionario_EntidadeCategoria )
    #print('---------------------------------------------------------------------------------------------------------------')
print("Total de itens - Entidades x Categorias:", len(dicionario_EntidadeCategoria))

Total de itens - Entidades x Categorias: 2906


In [9]:
# Convertendo os textos do diconario (campo Key) para letras minusculas
#dicionario_EntidadeCategoria = {k.lower(): v for k, v in dicionario_EntidadeCategoria.items()}

#OBS: a quantidade de dados do dicionario diminui porque a propria inserção dos dados de texto em minusculo remove os item 
# duplicados.
#len(dicionario_EntidadeCategoria)

In [10]:
# alterando o texto que no dicionario que está diferente

dicionario_EntidadeCategoria = { k.replace('Cachaça Doce Paixão.', 'Cachaça Doce Paixão'): v for k, v in dicionario_EntidadeCategoria.items()}
#dicionario_EntidadeCategoria

In [13]:
from requests.structures import CaseInsensitiveDict

dicionario_EntidadeCategoria = CaseInsensitiveDict(dicionario_EntidadeCategoria)
#dicionario_EntidadeCategoria

In [19]:
# Coloca os itens do dicionario em uma listas, assim cada item do dicionario vira uma tupla em uma posição da lista

listaDicionariosOrdenadosComEntidadesJuntos = list(dicionario_EntidadeCategoria.items())
#listaDicionariosOrdenadosComEntidadesJuntos

In [7]:
# O código cria uma lista a partir dos termos que compõem as chaves (keys) do dicionario 'dicionario_EntidadeCategoria'...
# Isso é necessário para que o código da linha abaixo funcione.

list_termos_Equivalentes_A_Entidade = list(dicionario_EntidadeCategoria)
len(list_termos_Equivalentes_A_Entidade)


# Modifica o caracter '|' por '||', porque logo mais a frente, haverá uma celula com código onde 
# essa lista com as entidades será transforma em strings sepradas por '|', para evitar erro na parte do código de rotulaçaõ
#semiautomatica é necessario substituir '|' por '||'

termos_Equivalentes_A_Entidade = []

for item in list_termos_Equivalentes_A_Entidade:
    boolean = re.search("\|", item)
    if boolean:
        item_Da_Lista = re.sub('\|', '_', item, count=1)
        termos_Equivalentes_A_Entidade.append(item_Da_Lista)
    else:
        termos_Equivalentes_A_Entidade.append(item)
        
#termos_Equivalentes_A_Entidade

Explicação do código abaixo:
Ordena os dados da lista de entidades criada no código acima em ordem decrescente. Isso é necessário para que o código principal de rotulação não gere overlaping (sobreposições), isso ocorre quando o token que representa a entidade é sobreposto, ou seja, quando a posição incial e final de uma entidade em um texto é sobreposta.
Por exemplo, dadas as seguintes entidades que representam nomes de bebidas: (1)'Cachaça Salinas' e (2)'Cachaça Salinas Tradicional'. E o texto: A Cachaça Salinas Tradicional é proveniente da cidade de Salinas/ MG.
O algoritmo principal de rotulação irá verificar iterativamente se o texto possui a entidade 'Cachaça Salinas Tradicional' ou a entidade 'Cachaça Salinas', e se a lista com os nomes de entidades não estiver ordenada na ordem decrescente e a entidade (1) que é menor vier primeiro na lista de entidades nomeadas, então o algoritmo através da função re.finditer(entidade,texto), irá encontrar a entidade (1) dentro da entidade (2) e criar uma rotulação (inicioEntidade:2; fimEntidade:17), mas depois quando o algoritmo iterar na entidade (2) irá sobrepor o que já havia rotulado antes, ou seja, irá gerar uma rotulação sopreposta (inicioEntidade:2; fimEntidade:29), pois a rotulaçõa da entidade (1) está dentro da rotulação da entidade (2). E os dados de treino do modelo de reconhecimento de entidade nomeada com esse tipo de sobreposição não é aceito pelo algoritmo do spaCy.

In [ ]:
lista_Com_NomesDeBebidas_Ordenados = sorted(termos_Equivalentes_A_Entidade,key=len,reverse=True)
len(lista_Com_NomesDeBebidas_Ordenados)

for item in lista_Com_NomesDeBebidas_Ordenados:
    print(item)

O código a seguir, converter a lista em um conjunto de string separadas pelo simbolo '|', para depois passar essa string como parâmetro na função do python 're.finditer'. A lista de entidades nomeadas, precisa ser tranformada em uma lista de string separadas pelo '|' para que seja usada na função principal de rotulação, na parte do 're.finditer(string_NomesBebidas, texto, re.I)' onde o regex, só aceita como paramêtro strings.

In [ ]:
string_NomesBebidas = '|'.join(map(str, lista_Com_NomesDeBebidas_Ordenados))
#string_NomesBebidas

### Código Principal de rotulação semiautomatica

o código da cela abaixo, consiste na função principal que recebe como entrada uma lista contendo em cada posição uma tupla 
(essa tupla guarda por sua vez um par de 'entidade versus token')
Por fim, esse função gera como saída uma lista 't'. Essa lista 't' contém em cada uma de suas posições dicionarios esse dicionarios por sua 

In [ ]:
def rotulacao_Entidades(dados_De_Texto_Para_Rotular): 
    t = []#list Final
    
    #Itera por cada linha da lista que contém os texto a serem rotulados
    for index, line in enumerate(dados_De_Texto_Para_Rotular):
        
        #variavel recebe uma lista de iteratos gerada pelo metodo re.finiter(), o qual procura cada entidade da...
        #listaDicionariosOrdenadosComEntidadesJuntos em cada texto da lista dados_De_Texto_Para_Rotular.
        # 'x[0]' é o texto/entidade que fica na primeira posição da tupla e 'x' e a tupla contendo entidade e categoria. 
        objetoIterator = [re.finditer(x[0],line,re.I) for x in listaDicionariosOrdenadosComEntidadesJuntos]
        
        for iterator in objetoIterator[:]:
            print(line)
            print(iterator.start())
            print(iterator.end())
        
            

In [39]:
def rotulacao_Entidades(dados_De_Texto_Para_Rotular): 
    t = []#list Final
    
    #Itera por cada linha da lista com texto a serem rotulados
    for i in dados_De_Texto_Para_Rotular:
        texto = i;
        listaL = [] #list Support
        print(texto)
        
        # Atribui a variavel 'resultado_Comparacao' um objeto do tipo 'callable_iterator', o qual contem informações dos matchs
        # ou seja, onde a entidade encontrado no texto inicia e finaliza bem como o grupo ao qual cada match pertence.
        resultado_Comparacao = re.finditer(string_NomesBebidas, texto, re.I)
        #print('match:', resultado_Comparacao)

        #Itera pelos objetos resultantes do match encontrado no texto para a entidade da lista de entidades em questão  
        for index, iterator in enumerate(resultado_Comparacao): 
            #adiciona da lista 
            listaL.append((iterator.start(), iterator.end()+1, dicionario_EntidadeCategoria[iterator.group()]))
            print(iterator)
            print(listaL)
            
        if listaL:
            t.append((texto,{'entities':listaL}))
            #print('Lista T:',t,'\n')
        else:
            t.append((texto, {'entities':listaL}))
    return t            

In [ ]:
# Atribuindo o resultado da função a uma variavel 

textos_Rotulados = rotulacao_Entidades(dados_De_Texto_Para_Rotular)

In [ ]:
# Verificar o resultado do algoritmo de rotrulação, tanto o tamanho quando os dados gerados

textos_Rotulados

### Salvando os as tuplas compostas pelo texto e pelo dicionario para utilizar no treinamento do modelo de REN

In [21]:
with open('C:\\Users\\prisc\\Downloads\\BaseResultadoRotulacaoSemiautomatica', "wb") as fp:
    
    #for tupla in textos_Rotulados:
        #print(tupla)
        pickle.dump(textos_Rotulados,fp)

In [123]:
# Testando para ver se é possivel abrir o arquivo salvo com uma lista de tuplas

with open('C:\\Users\\prisc\\Downloads\\Algoritmo_Criacao_Automatica_Corpus_Treino\\codigoPrincipal\\Dados_Para_Treinar_o_Modelo_De_Reconhecimento_De_Entidades\\Dados_Rotulados_Com_Categoria_RecipienteArmazenamento_TipoMadeira_TreinarModelo.txt', "rb") as fp:
    listas_De_Tuplas = pickle.load(fp)
    #print(listas_De_Tuplas)
    
#listas_De_Tuplas

#### Para verificar se a lista possui itens repetidos